In [1]:
# if run on colab
#!pip install torcheeg
#from google.colab import mount
#drive.mount('/content/drive', force_remount=True) 

In [2]:
# if run on kaggle
!pip install torcheeg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.5/214.5 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.8/230.8 kB 13.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.2/107.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.0 MB/s eta 0:00:00
  Created wheel for torcheeg: filename=torc

In [3]:
import torcheeg
from torcheeg import transforms
from torcheeg.datasets import BCICIV2aDataset
from torcheeg.model_selection import KFoldGroupbyTrial
from torch.utils.data import DataLoader
from torcheeg.models import ATCNet, EEGNet
import torch

from torcheeg.trainers import ClassifierTrainer

import pytorch_lightning as pl

In [4]:
# if run on kaggle
root_data_path = '/kaggle/input/bci-competition-iv-dataset-2a-in-mat-format/BCICIV-2a-mat'

# if run on colab
# root_data_path = './BCICIV-2a-mat/'

In [5]:
dataset = BCICIV2aDataset(
    root_path=root_data_path,
    io_path=f'./examples_pipeline/bciciv-2a',
    # skip_trial_with_artifacts=True,
    # offline_transform=transforms.Compose([
    #     transforms.BandDifferentialEntropy(apply_to_baseline=True),
    #     transforms.To2d(apply_to_baseline=True),
    #     transforms.ToTensor(apply_to_baseline=True)
    # ]),
    online_transform=transforms.Compose([
        # transforms.To2d(apply_to_baseline=True),
        # transforms.ToTensor(apply_to_baseline=True),
        transforms.To2d(),
        transforms.ToTensor(),
        # transforms.CWTSpectrum(apply_to_baseline=True),
        # transforms.BandDifferentialEntropy(apply_to_baseline=True),
        # transforms.BaselineRemoval(),
    ]),
    label_transform=transforms.Compose([
        transforms.Select('label'),
        transforms.Lambda(lambda x: x - 1)
    ]),
    chunk_size=7*250,
    num_worker=2
)

[2024-05-23 02:30:24] INFO (torcheeg/MainThread) 🔍 | Processing EEG data. Processed EEG data has been cached to ./examples_pipeline/bciciv-2a.
[2024-05-23 02:30:24] INFO (torcheeg/MainThread) ⏳ | Monitoring the detailed processing of a record for debugging. The processing of other records will only be reported in percentage to keep it clean.
[PROCESS]:   0%|          | 0/18 [00:00<?, ?it/s]
[RECORD /kaggle/input/bci-competition-iv-dataset-2a-in-mat-format/BCICIV-2a-mat/A04T.mat]: 0it [00:00, ?it/s]
[RECORD /kaggle/input/bci-competition-iv-dataset-2a-in-mat-format/BCICIV-2a-mat/A04T.mat]: 1it [00:01,  1.01s/it]
[RECORD /kaggle/input/bci-competition-iv-dataset-2a-in-mat-format/BCICIV-2a-mat/A04T.mat]: 16it [00:01, 19.26it/s]
[RECORD /kaggle/input/bci-competition-iv-dataset-2a-in-mat-format/BCICIV-2a-mat/A04T.mat]: 30it [00:01, 36.70it/s]
[RECORD /kaggle/input/bci-competition-iv-dataset-2a-in-mat-format/BCICIV-2a-mat/A04T.mat]: 42it [00:01, 50.38it/s]
[RECORD /kaggle/input/bci-competition

In [6]:
print("Dataset's info: ")
print(dataset.info)

Dataset's info: 
      start_at  end_at   clip_id subject_id  trial_id session subject  run  \
0          251    2001    A04T_0        A04         0       T     A04    1   
1         2254    4004    A04T_1        A04         1       T     A04    1   
2         4172    5922    A04T_2        A04         2       T     A04    1   
3         6124    7874    A04T_3        A04         3       T     A04    1   
4         8132    9882    A04T_4        A04         4       T     A04    1   
...        ...     ...       ...        ...       ...     ...     ...  ...   
5179     86751   88501  A04E_283        A04        43       E     A04    8   
5180     88657   90407  A04E_284        A04        44       E     A04    8   
5181     90585   92335  A04E_285        A04        45       E     A04    8   
5182     92699   94449  A04E_286        A04        46       E     A04    8   
5183     94758   96508  A04E_287        A04        47       E     A04    8   

      label  _record_id  
0         4   _recor

In [7]:
k_fold = KFoldGroupbyTrial(
    n_splits=10,
    split_path='./examples_pipeline/split',
    shuffle=True,
    random_state=44
)

In [8]:
DEVICE = 'gpu' if torch.cuda.is_available() else 'cpu'

for i, (train_dataset, val_dataset) in enumerate(k_fold.split(dataset)):
    train_loader = DataLoader(
        dataset=train_dataset,
        batch_size=64,
        shuffle=True,
        num_workers=4
    )
    val_loader = DataLoader(
        dataset=val_dataset,
        batch_size=64,
        shuffle=False,
        num_workers=4
    )
    
    model = ATCNet(
        num_classes=4,
        num_electrodes=22,
        chunk_size=7*250,
    )
    
    trainer = ClassifierTrainer(
        model=model,
        num_classes=4,
        lr=1e-4,
        weight_decay=1e-4,
        accelerator=DEVICE
    )

    trainer.fit(
        train_loader,
        val_loader, 
        max_epochs=250,
        default_root_dir=f'./examples_pipeline/atcnet_model/{i}',
        callbacks=[pl.callbacks.ModelCheckpoint(save_last=True)],
        enable_progress_bar=True,
        enable_model_summary=True,
        limit_val_batches=0.0
    )

    score = trainer.test(
        val_loader,
        enable_progress_bar=True,
        enable_model_summary=True
    )[0]
    print(f"Fold {i} test accuracy: {score['test_accuracy']: .4f}")

[2024-05-23 02:31:07] INFO (torcheeg/MainThread) 📊 | Create the split of train and test set.
[2024-05-23 02:31:07] INFO (torcheeg/MainThread) 😊 | Please set split_path to ./examples_pipeline/split for the next run, if you want to use the same setting for the experiment.
/opt/conda/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at /usr/local/src/pytorch/aten/src/ATen/native/Convolution.cpp:1008.)
  return F.conv2d(input, weight, bias, self.stride,
2024-05-23 02:31:18.531782: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-23 02:31:18.531917: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT 

Training: |          | 0/? [00:00<?, ?it/s]

[2024-05-23 02:31:36] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.384 train_accuracy: 0.277 

[2024-05-23 02:31:44] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.366 train_accuracy: 0.333 

[2024-05-23 02:31:53] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.352 train_accuracy: 0.351 

[2024-05-23 02:32:01] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.337 train_accuracy: 0.370 

[2024-05-23 02:32:10] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.323 train_accuracy: 0.400 

[2024-05-23 02:32:19] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.304 train_accuracy: 0.433 

[2024-05-23 02:32:27] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.286 train_accuracy: 0.458 

[2024-05-23 02:32:36] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.268 train_accuracy: 0.480 

[2024-05-23 02:32:44] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.249 train_accuracy: 0.510 

[2024-05-23 02:32:53] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.238 train_accuracy: 0.521 



Testing: |          | 0/? [00:00<?, ?it/s]

[2024-05-23 03:07:17] INFO (torcheeg/MainThread) 
[Test] test_loss: 1.088 test_accuracy: 0.628 



...

[2024-05-23 08:22:50] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.384 train_accuracy: 0.279 

[2024-05-23 08:22:59] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.361 train_accuracy: 0.338 

[2024-05-23 08:23:09] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.345 train_accuracy: 0.373 

[2024-05-23 08:23:18] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.326 train_accuracy: 0.404 

[2024-05-23 08:23:28] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.306 train_accuracy: 0.435 

[2024-05-23 08:23:38] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.289 train_accuracy: 0.462 

[2024-05-23 08:23:47] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.272 train_accuracy: 0.483 

[2024-05-23 08:23:57] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.256 train_accuracy: 0.502 

[2024-05-23 08:24:06] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.250 train_accuracy: 0.504 

[2024-05-23 08:24:16] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.233 train_accuracy: 0.525 



Testing: |          | 0/? [00:00<?, ?it/s]

[2024-05-23 09:02:36] INFO (torcheeg/MainThread) 
[Test] test_loss: 1.049 test_accuracy: 0.681 



┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.6805555820465088     │
│         test_loss         │    1.0486254692077637     │
└───────────────────────────┴───────────────────────────┘

Fold 9 test accuracy:  0.6806


In [9]:
!mkdir weights

In [10]:
atc_weights_path = './weights/atc_weights.pt'
torch.save(model.state_dict(), atc_weights_path)
# to load weight:
# model.load_state_dict(torch.load(atc_weights_path))